In [1]:
# Imports
from biocrnpyler import *
from genelet import *
from subsbml import System, createSubsystem, combineSystems, createNewSubsystem, createBasicSubsystem, SimpleModel, SimpleReaction
import numpy as np
import pylab as plt

from bokeh.layouts import row
from bokeh.io import export_png

import warnings
import libsbml
import bokeh.io
import bokeh.plotting
import bioscrape

## Cells B and P communicating with each other ##
Cell_B = Bistable Switch

Cell_P = Produce2N

External envioronment is rR1 reservoir set at 0

In [2]:
ss1 = createSubsystem ("SBML_Models/BiSwitch_CRN.xml")
ss2 = createSubsystem ("SBML_Models/rR1_external_reservoir.xml")

# Membrane 
mb1 = createSubsystem("SBML_Models/rR1_membrane1_detailed.xml", membrane = True)
### Cell 3 ###
cell_3 = System("cell_3")

cell_3.setInternal([ss1])
cell_3.setExternal([ss2])
cell_3.setMembrane(mb1)

cell_3_model = cell_3.getModel()
cell_3_model.writeSBML('SBML_Models/cell_3_model.xml')
print (cell_3)

The subsystem from SBML_Models/rR1_membrane1_detailed.xml has multiple compartments


In [3]:
ss2 = createSubsystem ("SBML_Models/rR1_external_reservoir.xml")
ss3 = createSubsystem ("Produce_2N/Prod2_CRN.xml")

# Membrane 
mb1 = createSubsystem("SBML_Models/rR1_membrane1_detailed.xml", membrane = True)
### Cell 4 ###
cell_4 = System("cell_4")

cell_4.setInternal([ss3])
cell_4.setExternal([ss2])
cell_4.setMembrane(mb1)

cell_4_model = cell_4.getModel()
cell_4_model.writeSBML('SBML_Models/cell_4_model.xml')

print(cell_4)

The subsystem from SBML_Models/rR1_membrane1_detailed.xml has multiple compartments


In [4]:
combined_cell_3_4 = createNewSubsystem()
combined_cell_3_4.combineSubsystems([cell_3_model, cell_4_model], verbose = True)


combined_cell_3_4.setSpeciesAmount('rna_rR1', 100, compartment = 'cell_3_external')
combined_cell_3_4.setSpeciesAmount('rna_rR1', 0, compartment = 'cell_4_external')

# Cell 3 set species amount
combined_cell_3_4.setSpeciesAmount('Core1_OFF', 6e3, compartment = 'cell_3_internal')
combined_cell_3_4.setSpeciesAmount('Core2_OFF', 4e3, compartment = 'cell_3_internal')
combined_cell_3_4.setSpeciesAmount('dna_dA2', 6e3, compartment = 'cell_3_internal')
combined_cell_3_4.setSpeciesAmount('dna_dA1', 6e3, compartment = 'cell_3_internal')
combined_cell_3_4.setSpeciesAmount('protein_RNAseH', 20, compartment = 'cell_3_internal')
combined_cell_3_4.setSpeciesAmount('protein_RNAP', 300, compartment = 'cell_3_internal')

# Cell 4 set species amount
combined_cell_3_4.setSpeciesAmount('Prod2_OFF', 4e3, compartment = 'cell_4_internal')
combined_cell_3_4.setSpeciesAmount('dna_dA1', 4e3, compartment = 'cell_4_internal')
combined_cell_3_4.setSpeciesAmount('protein_RNAseH', 20, compartment = 'cell_4_internal')
combined_cell_3_4.setSpeciesAmount('protein_RNAP', 300, compartment = 'cell_4_internal')
combined_cell_3_4.setSpeciesAmount('rna_rR1', 0, compartment = 'cell_4_internal')


combined_cell_3_4.writeSBML('Cell_3_4model.xml')

print(combined_cell_3_4.changeLog)

C:\Users\Jeremiah\anaconda3\lib\site-packages\subsbml-0.1-py3.7.egg\subsbml\Subsystem.py:602: UserWarning: Not all of the Subsystems being combined are in the same Compartment
C:\Users\Jeremiah\anaconda3\lib\site-packages\subsbml-0.1-py3.7.egg\subsbml\Subsystem.py:943: UserWarning: Annotated propensity updated to: <PropensityType> type=massaction k=0.00394  </PropensityType>
C:\Users\Jeremiah\anaconda3\lib\site-packages\subsbml-0.1-py3.7.egg\subsbml\Subsystem.py:943: UserWarning: Annotated propensity updated to: <PropensityType> type=massaction k=0.0696  </PropensityType>
C:\Users\Jeremiah\anaconda3\lib\site-packages\subsbml-0.1-py3.7.egg\subsbml\Subsystem.py:943: UserWarning: Annotated propensity updated to: <PropensityType> type=massaction k=38.857142857142854  </PropensityType>
C:\Users\Jeremiah\anaconda3\lib\site-packages\subsbml-0.1-py3.7.egg\subsbml\Subsystem.py:943: UserWarning: Annotated propensity updated to: <PropensityType> type=massaction k=10.0  </PropensityType>
C:\Users\

{'kb': 'kb_combined', 'kd': 'kd_combined'}


In [5]:
# Calling Names

X_id1 = combined_cell_3_4.getSpeciesByName('complex_Core1_ON').getId()
X_id2 = combined_cell_3_4.getSpeciesByName('Core1_OFF').getId()
X_id3 = combined_cell_3_4.getSpeciesByName('complex_Core2_ON').getId()
X_id4 = combined_cell_3_4.getSpeciesByName('Core2_OFF').getId()

X_id5 = combined_cell_3_4.getSpeciesByName('Prod2_OFF').getId()
X_id6 = combined_cell_3_4.getSpeciesByName('complex_Prod2_ON').getId()

print (X_id1)

complex_Core1_ON_biocrnpyler_894839


In [6]:
# Simulate with BioScrape
timepoints = np.linspace(0,28800,1000)
results_1,_ = combined_cell_3_4.simulateWithBioscrape(timepoints)
timepoints = timepoints/3600

# For label convenience
x = 'Time (hours)'
y = 'Concentration (uM)'

bokeh.io.output_notebook()
a = bokeh.plotting.figure(plot_width=400, plot_height=300, x_axis_label = x, y_axis_label=y)
a.circle(timepoints, results_1[X_id1], legend_label = "Core1 ON" , color = "blue")
a.circle(timepoints, results_1[X_id3], legend_label = "Core2 ON", color = "red")
a.legend.click_policy="hide"
a.legend.location="bottom_right"

b = bokeh.plotting.figure(plot_width=400, plot_height=300, x_axis_label = x, y_axis_label=y)
b.circle(timepoints, results_1[X_id5], legend_label = "Prod2 OFF" , color = "blue")
b.circle(timepoints, results_1[X_id6], legend_label = "Prod ON", color = "red")
b.legend.click_policy="hide"
b.legend.location="center_right"

bokeh.io.show(row(a, b))
warnings.filterwarnings("ignore")

C:\Users\Jeremiah\anaconda3\lib\site-packages\bioscrape\sbmlutil.py:208: UserWarning: Compartments, UnitDefintions, Events, and some other SBML model components are not recognized by bioscrape. Refer to the bioscrape wiki for more information.
  warnings.warn('Compartments, UnitDefintions, Events, and some other SBML model components are not recognized by bioscrape. ' +
C:\Users\Jeremiah\anaconda3\lib\site-packages\scipy\integrate\odepack.py:248: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
  warnings.warn(warning_msg, ODEintWarning)
odeint failed with mxstep=500...

Loading BokehJS ...

In [8]:
# Calling Names DNA and RNA #
# X_id7 = combined_cell_3_4.getSpeciesByName('rna_rR1').getId()
X_id8 = combined_cell_3_4.getSpeciesByName('rna_rR1', compartment="cell_3_external").getId()

print(X_id8)

ValueError: The species rna_rR1 not found.